# 1. Selenium이란?
* 웹브라우저를 자동으로 제어하는 라이브러리
* 본래 다양한 웹브라우저를 사용한 웹사이트 자동 테스트 도구
* 현재는 웹브라우저를 자동 제어하여 동적 웹사이트에서 정보를 가져오는 용도로 많이 활용

# 2. 정적웹사이트 vs 동적웹사이트
* 정적 웹 사이트(Static website): HTML 페이지 안에 웹 브라우저에 표시되는 모든 정보가 포함된 사이트
    * requests 라이브러리를 활용해서 정보 추출 가능
* 동적 웹 사이트(Dynamic website): Java Script를 사용하여 상황에 따라 필요한 정보를 불러와서 표시하는 웹사이트
    * 상황에 맞추어 자바스크립트를 활용해 서버에 정보를 다시 요청하는 형식이라 requests로는 정보 추출이 어려움
    * selenium으로 웹브라우저를 조작해 원하는 정보가 다 표시된 후 정보 추출 필요

# 3. Selenium 웹사이트
* https://www.selenium.dev/documentation/webdriver/

# 4. Selenium 설치
* pip install selenium # 셀레니움 설치
* pip install webdriver-manager # web 드라이버 자동 다운로드
(아나콘다프롬프트는 pip말고 conda)
* testenv들어가서!!(conda activate testenv)

In [1]:
import selenium
print(selenium.__version__)

4.12.0


In [2]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import time

import yes24_dbio_copy as ydb

In [3]:
# 웹 드라이버를 버전에 맞게 자동으로 다운받고 옵션을 추가해 줌
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.set_window_size(1920, 1080) # 웹 브라우저 해상도 조절
driver.get('https://www.yes24.com/Product/Category/BestSeller') # 크롤링할 웹사이트 주소
driver.find_element(By.CSS_SELECTOR, 'option[value=\'120\']').click() # 120개씩 보기

In [ ]:
#lnb > div > div._lnbSearch_lnb_search_29ynI > div > div > form > input

In [14]:
driver.find_element(By.CSS_SELECTOR, 'a[title="2"]')


<selenium.webdriver.remote.webelement.WebElement (session="dd1887c175f9d555fa51b92e7f86e343", element="C3C08BA12CDA2DB995BD6835D3E70B32_element_188")>

자료수집!!!!! 페이지 전체 불러오는거 자료 수집!!!!

In [4]:
all_list_book =[] 

for i in  range(2,4):
    page_html = driver.page_source
    soup = bs(page_html, 'lxml')
    all_list_book.append(soup)
    time.sleep(5)
    
    driver.find_element(By.CSS_SELECTOR,f"a[title='{i}']").click()
driver.close()

In [5]:
len(all_list_book)

2

자료추출!!!!! li만 따로 불러와서 자료 빼내는거!!

In [1]:
yy = 2023
mon = 8

for j in all_list_book:
    book_list = j.select("#yesBestList > li")
    result = ydb.extract_bookinfo(yy, mon, j)

NameError: name 'all_list_book' is not defined

In [47]:
df["링크"]

0      https://www.yes24.com//Product/Goods/122532130
1      https://www.yes24.com//Product/Goods/122091079
2      https://www.yes24.com//Product/Goods/122090075
3      https://www.yes24.com//Product/Goods/122091168
4      https://www.yes24.com//Product/Goods/122090360
                            ...                      
115    https://www.yes24.com//Product/Goods/119627529
116    https://www.yes24.com//Product/Goods/122141963
117    https://www.yes24.com//Product/Goods/122483002
118    https://www.yes24.com//Product/Goods/115475489
119    https://www.yes24.com//Product/Goods/109308932
Name: 링크, Length: 120, dtype: object

In [8]:
import pandas as pd 


8월 베스트 셀러 1-3페이지 목록의 세부정보 링크로 세부정보 수집

In [9]:
result2 = pd.DataFrame()

df = pd.DataFrame(result)

detail_page_data = ydb.detail_page_info(df['링크'])
df['book_id'] = df['링크'].apply(lambda x: x.split("/")[-1])
detail_df = pd.DataFrame(detail_page_data)

result2 = pd.concat([result2,df])
final_result_df = pd.merge(result2, detail_df, how='inner', on="book_id")
display(final_result_df.head(2))
final_result_df.info()

0/120 데이터 추출 중
1/120 데이터 추출 중
2/120 데이터 추출 중
3/120 데이터 추출 중
4/120 데이터 추출 중
5/120 데이터 추출 중
6/120 데이터 추출 중
7/120 데이터 추출 중
8/120 데이터 추출 중
9/120 데이터 추출 중


,년,월,제목,부제목,링크,작가,출판사,출판일,가격,리뷰,리뷰링크,별점,태그,book_id,페이지,무게,사이즈,카테고리,책소개,출판사리뷰
0,2023,8,오늘도 나아가는 중입니다,세상과 소통하고 싶은 그녀의 생생하고 진솔한 이야기,https://www.yes24.com//Product/Goods/122532130,[조민],참새책방,2023년 09월,"15,120원",15,https://www.yes24.com/Product/Goods/122532130?...,10.0,[],122532130,256쪽,274g,125*188*15mm,"[에세이, 명사/연예인 에세이 , 한국 에세이, 국내도서]","\n\r\n“제 이름은 조민입니다.”사회적 논란의 중심에서누구의 딸이 아닌,‘조민’...","\n\n\r\n“제 이름은 조민입니다.”사회적 논란의 중심에서누구의 딸이 아닌,‘조..."
1,2023,8,디케의 눈물,대한검국에 맞선 조국의 호소,https://www.yes24.com//Product/Goods/122091079,[조국],다산북스,2023년 08월,"16,920원",16,https://www.yes24.com/Product/Goods/122091079?...,9.3,"[#법은정의로운가, #법을바라보는법]",122091079,336쪽,518g,140*210*20mm,"[사회 정치, 한국사회비평, 국내도서, 에세이, 사회비평/비판, 명사/연예인 에세이 ]",\n\r\n“등에 화살이 꽂힌 채 길 없는 길을 묵묵히 걸어가겠다.”\r\n법대 교...,\n\n\r\n“등에 화살이 꽂힌 채 길 없는 길을 묵묵히 걸어가겠다.”\r\n법대...


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   년        10 non-null     int64 
 1   월        10 non-null     int64 
 2   제목       10 non-null     object
 3   부제목      5 non-null      object
 4   링크       10 non-null     object
 5   작가       10 non-null     object
 6   출판사      10 non-null     object
 7   출판일      10 non-null     object
 8   가격       10 non-null     object
 9   리뷰       10 non-null     object
 10  리뷰링크     10 non-null     object
 11  별점       10 non-null     object
 12  태그       10 non-null     object
 13  book_id  10 non-null     object
 14  페이지      10 non-null     object
 15  무게       10 non-null     object
 16  사이즈      10 non-null     object
 17  카테고리     10 non-null     object
 18  책소개      10 non-null     object
 19  출판사리뷰    10 non-null     object
dtypes: int64(2), object(18)
memory usage: 1.6+ KB


In [11]:
detail_df

,book_id,페이지,무게,사이즈,카테고리,책소개,출판사리뷰
0,122532130,256쪽,274g,125*188*15mm,"[에세이, 명사/연예인 에세이 , 한국 에세이, 국내도서]","\n\r\n“제 이름은 조민입니다.”사회적 논란의 중심에서누구의 딸이 아닌,‘조민’...","\n\n\r\n“제 이름은 조민입니다.”사회적 논란의 중심에서누구의 딸이 아닌,‘조..."
1,122091079,336쪽,518g,140*210*20mm,"[사회 정치, 한국사회비평, 국내도서, 에세이, 사회비평/비판, 명사/연예인 에세이 ]",\n\r\n“등에 화살이 꽂힌 채 길 없는 길을 묵묵히 걸어가겠다.”\r\n법대 교...,\n\n\r\n“등에 화살이 꽂힌 채 길 없는 길을 묵묵히 걸어가겠다.”\r\n법대...
2,122090075,768쪽,836g,128*188*40mm,"[일본 장편소설, 소설/시/희곡, 일본소설, 국내도서]","\n\r\n첫 발표 이후 43년, 마음에 품어왔던 소설을 마침내 완성하다.\r\n하...","\n\n\r\n첫 발표 이후 43년, 마음에 품어왔던 소설을 마침내 완성하다.\r\..."
3,122091168,256쪽,562g,170*230*15mm,"[집/살림, 가정 살림, 인테리어, 국내도서]","\n\r\n오래되어 너무 낡았거나 효율이 떨어지는 집,가족의 라이프 스타일과 맞지 ...","\n\n\r\n오래되어 너무 낡았거나 효율이 떨어지는 집,가족의 라이프 스타일과 맞..."
4,122090360,292쪽,476g,145*210*18mm,"[성공학/경력관리, 자기계발, 국내도서]","\n\r\n‘미래의 나’를 적용하는 과학 분야 세계 최고의 전문가,자기계발 분야 파...","\n\n\r\n‘미래의 나’를 적용하는 과학 분야 세계 최고의 전문가,자기계발 분야..."
5,122455740,196쪽,538g,153*217*20mm,"[3-4학년 만화일반, 1-2학년, 3-4학년, 1-2학년 만화일반, 3-4학년 애...",\n\r\n유튜브 261만 구독 돌파! 인기 크리에이터 흔한남매웃음에 겨워 춤이 절...,\n\n\r\n유튜브 261만 구독 돌파! 인기 크리에이터 흔한남매웃음에 겨워 춤이...
6,117014613,736쪽,"1,030g",153*224*40mm,"[성공학/경력관리, 자기계발, 처세술/삶의 자세, 국내도서]",\n\r\n재야의 명저 『세이노의 가르침』 2023년판 정식 출간!순자산 천억 원대...,\n\n\r\n재야의 명저 『세이노의 가르침』 2023년판 정식 출간!순자산 천억 ...
7,118579613,464쪽,574g,135*215*23mm,"[사회학일반, 주제로 읽는 심리학, 사회 정치, 심리, 관계, 사회학, 인문, 국내...","\n\r\n아마존, 월 스트리트 저널 올해의 책\r\n애덤 그랜트·수전 케인·힐러리...","\n\n\r\n아마존, 월 스트리트 저널 올해의 책\r\n애덤 그랜트·수전 케인·힐..."
8,121812955,316쪽,512g,145*215*18mm,"[경제 경영, 성공학/경력관리, CEO/비즈니스맨, 국내도서]","\n\r\n출간 즉시 교보, 예스24, 알라딘 종합 1위네이버 블로그 경제·주식 분...","\n\n\r\n출간 즉시 교보, 예스24, 알라딘 종합 1위네이버 블로그 경제·주식..."
9,121946952,760쪽,0,152*225*40mm,"[기업/경영자스토리, CEO/비즈니스맨, 경영, 국내도서, 경제 경영]","\n\r\n“미래는 꿈꾸는 것이 아니라 만드는 것,\r\n그가 상상하면 모두 현실이...","\n\n\r\n“미래는 꿈꾸는 것이 아니라 만드는 것,\r\n그가 상상하면 모두 현..."


In [10]:
df

,년,월,제목,부제목,링크,작가,출판사,출판일,가격,리뷰,리뷰링크,별점,태그,book_id
0,2023,8,오늘도 나아가는 중입니다,세상과 소통하고 싶은 그녀의 생생하고 진솔한 이야기,https://www.yes24.com//Product/Goods/122532130,[조민],참새책방,2023년 09월,"15,120원",15,https://www.yes24.com/Product/Goods/122532130?...,10.0,[],122532130
1,2023,8,디케의 눈물,대한검국에 맞선 조국의 호소,https://www.yes24.com//Product/Goods/122091079,[조국],다산북스,2023년 08월,"16,920원",16,https://www.yes24.com/Product/Goods/122091079?...,9.3,"[#법은정의로운가, #법을바라보는법]",122091079
2,2023,8,도시와 그 불확실한 벽,None,https://www.yes24.com//Product/Goods/122090075,[무라카미 하루키],문학동네,2023년 09월,"17,550원",30,https://www.yes24.com/Product/Goods/122090075?...,8.8,"[#문학뉴스레터에소개된책, #이달의굿즈, #하루키신드롬, #하루키월드, #오랜만이에...",122090075
3,2023,8,일생에 한 번 내 집을 고친다면,삶이 가벼워지는 미니멀 인테리어,https://www.yes24.com//Product/Goods/122091168,[오아시스(김혜정)],터치아트,2023년 09월,"19,800원",8,https://www.yes24.com/Product/Goods/122091168?...,9.9,"[#셀프인테리어, #미니멀라이프, #그래제본소]",122091168
4,2023,8,퓨처 셀프,현재와 미래가 달라지는 놀라운 혁명,https://www.yes24.com//Product/Goods/122090360,[벤저민 하디],상상스퀘어,2023년 08월,"17,820원",5,https://www.yes24.com/Product/Goods/122090360?...,8.4,[#멘토를찾아서],122090360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2023,8,나는 어떻게 삶의 해답을 찾는가,고명환의 독서 내공,https://www.yes24.com//Product/Goods/119627529,[고명환],라곰,2023년 06월,"16,020원",39,https://www.yes24.com/Product/Goods/119627529?...,9.6,[],119627529
116,2023,8,대화력의 비밀,유연한 인생을 위한 36가지 대화의 기술,https://www.yes24.com//Product/Goods/122141963,[황시투안],미디어숲,2023년 09월,"16,020원",11,https://www.yes24.com/Product/Goods/122141963?...,9.3,[],122141963
117,2023,8,윤찐빵의 생존일지,모락모락 차가운,https://www.yes24.com//Product/Goods/122483002,[윤찐빵],사회평론아카데미,2023년 09월,"15,120원",0,None,0,[],122483002
118,2023,8,키위엔 영어회화 하루 5분의 기적,기초도 3개월이면 회화가 되는 특허 받은 영어 학습법,https://www.yes24.com//Product/Goods/115475489,[박강준],키위엔,2022년 12월,"17,550원",23,https://www.yes24.com/Product/Goods/115475489?...,9.9,[],115475489


In [36]:
df

,년,월,제목,부제목,링크,작가,출판사,출판일,가격,리뷰,리뷰링크,별점,태그
0,2023,8,오늘도 나아가는 중입니다,세상과 소통하고 싶은 그녀의 생생하고 진솔한 이야기,https://www.yes24.com//Product/Goods/122532130,[조민],참새책방,2023년 09월,"15,120원",15,https://www.yes24.com/Product/Goods/122532130?...,10.0,[]
1,2023,8,디케의 눈물,대한검국에 맞선 조국의 호소,https://www.yes24.com//Product/Goods/122091079,[조국],다산북스,2023년 08월,"16,920원",16,https://www.yes24.com/Product/Goods/122091079?...,9.3,"[#법은정의로운가, #법을바라보는법]"
2,2023,8,도시와 그 불확실한 벽,None,https://www.yes24.com//Product/Goods/122090075,[무라카미 하루키],문학동네,2023년 09월,"17,550원",30,https://www.yes24.com/Product/Goods/122090075?...,8.8,"[#문학뉴스레터에소개된책, #이달의굿즈, #하루키신드롬, #하루키월드, #오랜만이에..."
3,2023,8,일생에 한 번 내 집을 고친다면,삶이 가벼워지는 미니멀 인테리어,https://www.yes24.com//Product/Goods/122091168,[오아시스(김혜정)],터치아트,2023년 09월,"19,800원",8,https://www.yes24.com/Product/Goods/122091168?...,9.9,"[#셀프인테리어, #미니멀라이프, #그래제본소]"
4,2023,8,퓨처 셀프,현재와 미래가 달라지는 놀라운 혁명,https://www.yes24.com//Product/Goods/122090360,[벤저민 하디],상상스퀘어,2023년 08월,"17,820원",5,https://www.yes24.com/Product/Goods/122090360?...,8.4,[#멘토를찾아서]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2023,8,나는 어떻게 삶의 해답을 찾는가,고명환의 독서 내공,https://www.yes24.com//Product/Goods/119627529,[고명환],라곰,2023년 06월,"16,020원",39,https://www.yes24.com/Product/Goods/119627529?...,9.6,[]
116,2023,8,대화력의 비밀,유연한 인생을 위한 36가지 대화의 기술,https://www.yes24.com//Product/Goods/122141963,[황시투안],미디어숲,2023년 09월,"16,020원",11,https://www.yes24.com/Product/Goods/122141963?...,9.3,[]
117,2023,8,윤찐빵의 생존일지,모락모락 차가운,https://www.yes24.com//Product/Goods/122483002,[윤찐빵],사회평론아카데미,2023년 09월,"15,120원",0,None,0,[]
118,2023,8,키위엔 영어회화 하루 5분의 기적,기초도 3개월이면 회화가 되는 특허 받은 영어 학습법,https://www.yes24.com//Product/Goods/115475489,[박강준],키위엔,2022년 12월,"17,550원",23,https://www.yes24.com/Product/Goods/115475489?...,9.9,[]


In [29]:
df

,년,월,제목,부제목,링크,작가,출판사,출판일,가격,리뷰,리뷰링크,별점,태그
0,2023,8,오늘도 나아가는 중입니다,세상과 소통하고 싶은 그녀의 생생하고 진솔한 이야기,https://www.yes24.com//Product/Goods/122532130,[조민],참새책방,2023년 09월,"15,120원",15,https://www.yes24.com/Product/Goods/122532130?...,10.0,[]
1,2023,8,디케의 눈물,대한검국에 맞선 조국의 호소,https://www.yes24.com//Product/Goods/122091079,[조국],다산북스,2023년 08월,"16,920원",16,https://www.yes24.com/Product/Goods/122091079?...,9.3,"[#법은정의로운가, #법을바라보는법]"
2,2023,8,도시와 그 불확실한 벽,None,https://www.yes24.com//Product/Goods/122090075,[무라카미 하루키],문학동네,2023년 09월,"17,550원",30,https://www.yes24.com/Product/Goods/122090075?...,8.8,"[#문학뉴스레터에소개된책, #이달의굿즈, #하루키신드롬, #하루키월드, #오랜만이에..."
3,2023,8,일생에 한 번 내 집을 고친다면,삶이 가벼워지는 미니멀 인테리어,https://www.yes24.com//Product/Goods/122091168,[오아시스(김혜정)],터치아트,2023년 09월,"19,800원",8,https://www.yes24.com/Product/Goods/122091168?...,9.9,"[#셀프인테리어, #미니멀라이프, #그래제본소]"
4,2023,8,퓨처 셀프,현재와 미래가 달라지는 놀라운 혁명,https://www.yes24.com//Product/Goods/122090360,[벤저민 하디],상상스퀘어,2023년 08월,"17,820원",5,https://www.yes24.com/Product/Goods/122090360?...,8.4,[#멘토를찾아서]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2023,8,나는 어떻게 삶의 해답을 찾는가,고명환의 독서 내공,https://www.yes24.com//Product/Goods/119627529,[고명환],라곰,2023년 06월,"16,020원",39,https://www.yes24.com/Product/Goods/119627529?...,9.6,[]
116,2023,8,대화력의 비밀,유연한 인생을 위한 36가지 대화의 기술,https://www.yes24.com//Product/Goods/122141963,[황시투안],미디어숲,2023년 09월,"16,020원",11,https://www.yes24.com/Product/Goods/122141963?...,9.3,[]
117,2023,8,윤찐빵의 생존일지,모락모락 차가운,https://www.yes24.com//Product/Goods/122483002,[윤찐빵],사회평론아카데미,2023년 09월,"15,120원",0,None,0,[]
118,2023,8,키위엔 영어회화 하루 5분의 기적,기초도 3개월이면 회화가 되는 특허 받은 영어 학습법,https://www.yes24.com//Product/Goods/115475489,[박강준],키위엔,2022년 12월,"17,550원",23,https://www.yes24.com/Product/Goods/115475489?...,9.9,[]


In [27]:
result

[{'년': 2023,
  '월': 8,
  '제목': '오늘도 나아가는 중입니다',
  '부제목': '세상과 소통하고 싶은 그녀의 생생하고 진솔한 이야기',
  '링크': 'https://www.yes24.com//Product/Goods/122532130',
  '작가': ['조민'],
  '출판사': '참새책방',
  '출판일': '2023년 09월',
  '가격': '15,120원',
  '리뷰': '15',
  '리뷰링크': 'https://www.yes24.com/Product/Goods/122532130?ReviewYn=Y',
  '별점': '10.0',
  '태그': []},
 {'년': 2023,
  '월': 8,
  '제목': '디케의 눈물',
  '부제목': '대한검국에 맞선 조국의 호소',
  '링크': 'https://www.yes24.com//Product/Goods/122091079',
  '작가': ['조국'],
  '출판사': '다산북스',
  '출판일': '2023년 08월',
  '가격': '16,920원',
  '리뷰': '16',
  '리뷰링크': 'https://www.yes24.com/Product/Goods/122091079?ReviewYn=Y',
  '별점': '9.3',
  '태그': ['#법은정의로운가', '#법을바라보는법']},
 {'년': 2023,
  '월': 8,
  '제목': '도시와 그 불확실한 벽',
  '부제목': None,
  '링크': 'https://www.yes24.com//Product/Goods/122090075',
  '작가': ['무라카미 하루키'],
  '출판사': '문학동네',
  '출판일': '2023년 09월',
  '가격': '17,550원',
  '리뷰': '30',
  '리뷰링크': 'https://www.yes24.com/Product/Goods/122090075?ReviewYn=Y',
  '별점': '8.8',
  '태그': ['#문학뉴스레터에소개된책', '#이달의굿즈

In [ ]:
# y = 0
# y_step = 1000
# for i in range(1,3):
#     y = y + ystep
#     script = f'window.scrollTo({0},{y})'
#     driver.execute_script(script)
#     time.sleep(3)
# page_html = driver.page_source
# soup = bs(page_html, 'lxml')
# all_booklist.append(soup)
# driver.find_element(By.CSS_SELECTOR, 'div.bSGoodsPagen > div > div > div > {'num'}').click() # 다음페이지 넘김 버튼 클릭 
# time.sleep(5)

In [ ]:
#bestContentsWrap > div.bSGoodsPagen > div > div > div > a:nth-child(1)

네이버 쇼핑 크롤링 액세스 거부당한거

In [ ]:
# 네이버 쇼핑 검색창에 파이썬 넣는거
# search_box = driver.find_element(By.CSS_SELECTOR, 'div._lnbSearch_lnb_search_29ynI > div > div > form > input')
# search_box.send_keys('파이썬' + Keys.ENTER)

In [11]:
# all_booklist = []
# for page in range(1,20):
#     y = 0 # 마우스 스크롤 시작 지점
#     y_step = 1000 # 마우스 스크롤 길이
#     for i in range(1,8):
#         y = y + y_step
#         script = f'window.scrollTo({0},{y})' # 자바 스크립트로 위치 지정
#         driver.execute_script(script)        # 스크롤 실행
#         time.sleep(2)
#     page_html = driver.page_source # 페이지 끝에서 40개의 책이 로딩된 결과를 저장
#     soup = bs(page_html, 'lxml')
#     all_booklist.append(soup)
#     driver.find_element(By.CSS_SELECTOR, 'div.bSGoodsPagen > div > div > div > {'num'}').click() # 다음페이지 넘김 버튼 클릭 
#     time.sleep(2)